# Project 6e - Advection (this is optional - work on it if you're finished with 6e, but it's okay if you don't get here)

Recall that our general form for a PDE is that of a conservation law: namely that 
$$
\frac{\partial u}{\partial t} + \frac{\partial F}{\partial x} = 0
$$
where $F$ is a flux.  For the diffusion equation, we assumed that the flux was proportional to the imbalance in stuff (heat, people, whatever) in adjacent locations.  However, there are other types of fluxes.  For example, consider the case in which a material is being carried along by some background flow: birds or volcanic ash plumes being carried along by the wind or similar.  Such a flux has a deceptively simple form
$$
F = v u
$$
where $v$ is a velocity (which we'll assume to be constant in this case).  The resulting equation is 
$$
\frac{\partial u}{\partial t} + v \frac{\partial u}{\partial x} = 0.
$$
This PDE is called the *advection equation*.  the difference between this and the diffusion equation is that it involves a first spatial derivative rather than second.  To solve this, we can try to use similar methods.  The centered first finite difference looks like
$$
\frac{\partial u_i}{\partial x} \approx \begin{bmatrix} -\frac{1}{2 \Delta x} & 0 & \frac{1}{2 \Delta x} \end{bmatrix} \begin{bmatrix} u_{i-1} \\ u_i \\ u_{i+1} \end{bmatrix}.
$$
We can implement this fairly simply by replacing the formula for $A$ in the diffusion equation with the discretization given above.  There are a few subtleties: first, because we've only got a first derivative, this equation only allows one boundary condition.  For simplicity, we'll set a Dirichlet condition of one on the left side.  However, since we're using a centered finite difference, our rightmost point still requires some special treatment because it relies on a point outside the domain.  To fix this problem, we'll simply replace that row with the non-centered finite difference
$$
\frac{\partial u_{n-1}}{\partial x} \approx \begin{bmatrix} -\frac{1}{\Delta x} & \frac{1}{\Delta x} \end{bmatrix} \begin{bmatrix} u_{n-2} \\ u_{n-1} \end{bmatrix}.
$$
We can see these operations in the code below.  

In [ ]:
import numpy as np
import ode_methods as om
import matplotlib.pyplot as plt

class Advection:
    
    def __init__(self,x,v=0.1): 
        self.v = v
        self.x = x
        self.dx = dx = x[1]-x[0]
        self.n = len(x)
        
        self.A_adv = np.zeros((self.n,self.n))
        for i in range(1,self.n-1):
            self.A_adv[i,i-1] = -v/(2*dx)
            self.A_adv[i,i+1] = v/(2*dx)
        
        self.A_adv[-1,-1] = v/dx
        self.A_adv[-1,-2] = -v/dx
            
        self.A = -self.A_adv
        
        self.b = np.zeros(self.n)
        
    def rhs(self,t,u):
        dudt = self.A @ u + self.b
        return dudt
    
    def stiffness_matrix(self):
        return self.A 
    
    def load_vector(self):
        return self.b

We can integrate using Backward Euler (note the way that I am handling Dirichlet BCs here - I prefer this method)

In [ ]:
class BackwardEuler:
    def __init__(self,dirichlet_bcs=[(0,1.0)]):
        self.dirichlet_bcs = dirichlet_bcs
    
    def step(self,ode,t,dt,u_0):
        f = u_0/dt + ode.load_vector()
        I = np.eye(len(u_0))
        L = I/dt - ode.stiffness_matrix()
        for index,value in self.dirichlet_bcs:
            L[index] = 0
            L[index,index] = 1
            f[index] = value
        u_1 = np.linalg.solve(L,f)
        return u_1

We'll use a zero initial condition across the domain, and integrate per usual.  

In [ ]:
x = np.linspace(0,1,101)

u0 = np.zeros_like(x)
method = BackwardEuler(dirichlet_bcs=[(0,1.0)])
advection = Advection(x,v=0.1)
integrator = om.Integrator(advection,method)
t,u = integrator.integrate([0,5],0.01,u0)

Plotting our solution, we get

In [ ]:
plt.plot(x,u[::100].T)

Oh no, wiggles!  This is clearly wrong - if we are simply carrying some material with constant velocity across the domain, the analytical solution should be $u(x,t) = H(x-vt)$, where $H$ is the step (or Heaviside) function.  Where are these wiggles coming from?  Consider the flow of information in this problem: if material is being transported into a cell from upwind, it shouldn't depend on what's going on downwind.  Yet investigation our centered finite difference, it's clear that it does, and this is what is leading to the oscillations in the solution.  To fix this problem, we need to align our numerical scheme with our intuition, and use an approximation to the first derivative that only relies on the cell itself and its upwind neighbors.  Because velocity in this problem is moving from left to right, a simple method for this is
$$
\frac{\partial u_{i}}{\partial x} \approx \begin{bmatrix} -\frac{1}{\Delta x} & \frac{1}{\Delta x} \end{bmatrix} \begin{bmatrix} u_{i-1} \\ u_{i} \end{bmatrix}.
$$
(Note that this is the same method we already applied at the right boundary).  This method is called 'upwinding'.  **Implement upwinding and show that it eliminates the spurious oscillations**.  However, this improvement comes at a cost, in that the non-centered difference formula is only first order accurate.  **How does this inaccuracy manifest itself in the solution**?  Compare your result to the analytical (step function) solution.  See how this equation behaves with different initial and boundary conditions.  

As a last point, there can be both advective and diffusive fluxes present in a system.  Devise and implement a method for solving the advection-diffusion equation
$$
\frac{\partial u}{\partial t} + v \frac{\partial u}{\partial x} = k\frac{\partial^2 u}{\partial x^2}
$$